In [1]:
sample_in1 = """
#############
#...........#
###B#C#B#D###
  #A#D#C#A#
  #########
"""

my_in = """
#############
#...........#
###C#C#B#D###
  #D#A#B#A#
  #########
"""

"""Graph:

0 - 1 - x - 3 - x - 5 - x - 7 - x - 9 - 10
        |       |       |       |
      Room1   Room2   Room3   Room4
        |       |       |       |
      Room1b  Room2b  Room3b  Room4b

can't stop at X spaces, 
"""

MOVE_COST = {
    'A': 1,
    'B': 10,
    'C': 100,
    'D': 1000,
}
MOVEMENTS = {  # Room -> Hallway -> steps
    'A': {0: [1, 2], 1: [2], 3: [2], 5: [2, 3, 4], 7: [2, 3, 4, 5, 6],
          9: [2, 3, 4, 5, 6, 7, 8], 10: [2, 3, 4, 5, 6, 7, 8, 9]},
    'B': {0: [1, 2, 3, 4], 1: [2, 3, 4], 3: [4], 5: [4], 7: [4, 5, 6],
          9: [4, 5, 6, 7, 8], 10: [4, 5, 6, 7, 8, 9]},
    'C': {0: [1, 2, 3, 4, 5, 6], 1: [2, 3, 4, 5, 6], 3: [4, 5, 6], 5: [6],
          7: [6], 9: [6, 7, 8], 10: [6, 7, 8, 9]},
    'D': {0: [1, 2, 3, 4, 5, 6, 7, 8], 1: [2, 3, 4, 5, 6, 7, 8],
          3: [4, 5, 6, 7, 8], 5: [6, 7, 8], 7: [8], 9: [8], 10: [8, 9]},
}


In [2]:
import re


def get_input(s):
    lines = s.strip().split('\n')
    first_line = re.search('#([ABCD])#([ABCD])#([ABCD])#([ABCD])#', lines[2])
    second_line = re.search('#([ABCD])#([ABCD])#([ABCD])#([ABCD])#', lines[3])
    fl = first_line.groups()
    sl = second_line.groups()
    rooms = zip(fl, sl)
    return tuple(rooms)


print(get_input(sample_in1))
print(get_input(my_in))


(('B', 'A'), ('C', 'D'), ('B', 'C'), ('D', 'A'))
(('C', 'D'), ('C', 'A'), ('B', 'B'), ('D', 'A'))


In [3]:
def prob1(s):
    # We want AA in Room1, BB in room2, CC in room3 and DD in room4
    rooms = get_input(s)
    status = {'A': list(rooms[0]), 'B': list(rooms[1]),
              'C': list(rooms[2]), 'D': list(rooms[3]),
              'H': {}}
    result = get_min_cost(status)
    print(f'Result {rooms}: {result}')
    return result


def is_room_ok(status, room):
    return all([(r == room or not r) for r in status[room]])


assert is_room_ok({'A': [None, None]}, 'A') == True  # An empty stack is ok
assert is_room_ok({'A': ['B', None]}, 'A') == False  # Only foraigners: not ok
assert is_room_ok({'A': [None, 'B']}, 'A') == False  # Only foraigners: not ok
assert is_room_ok({'A': ['A', 'A']}, 'A') == True  # Only citizens: ok
assert is_room_ok({'A': ['A', None]}, 'A') == True  # Only citizens: ok
assert is_room_ok({'A': [None, 'A']}, 'A') == True  # Only citizens: ok
assert is_room_ok({'A': ['A', 'B']}, 'A') == False  # A mixed stack is NOT ok


def free_way(status, hpos, room):
    for cell in MOVEMENTS[room][hpos]:
        if cell in status['H']:
            return False
    return True


def get_moves(status):
    result = []
    for hpos, val in status['H'].items():
        if is_room_ok(status, val) and free_way(status, hpos, val):
            # Move from H to Room {val}
            result.append(('H', val, hpos))
            return result  # shortcut, this is a no-brainer
    for room in "ABCD":
        if not is_room_ok(status, room):
            # Move from room to at any position available in H
            for hpos in MOVEMENTS[room]:
                if free_way(status, hpos, room) and hpos not in status['H']:
                    result.append((room, 'H', hpos))
    return result


def apply_move(statuskey, move, stack_depth):
    status = key_to_dict(statuskey)
    from_room, to_room, hpos = move
    if 'H' == from_room:
        # Add to room and remove it from hpos
        room = to_room
        val = room
        room_position = _get_room_empty_depth(status, room, stack_depth)
        assert status['H'][hpos] == val, (hpos, room, status)
        del status['H'][hpos]
        status[room][room_position] = room
    elif 'H' == to_room:
        # Remove from room and put it in hpos
        room = from_room
        room_position = _get_room_empty_depth(status, room, stack_depth) + 1
        val = status[room][room_position]
        status[room][room_position] = None
        assert hpos not in status['H'], (hpos, room, status)
        status['H'][hpos] = val
    else:
        assert False, move
    movements = (len(MOVEMENTS[room][hpos])) + room_position + 1
    move_cost = movements * MOVE_COST[val]
    return status, move_cost

def _get_room_empty_depth(status, room, stackdepth):
    for i in range(stackdepth):
        if status[room][i] is not None:
            return i - 1
    return i


def dict_to_key(status):
    sorted_h_keys = sorted(status['H'].keys())
    return (
        tuple(status['A']), tuple(status['B']),
        tuple(status['C']), tuple(status['D']),
        tuple([(k, status['H'][k]) for k in sorted_h_keys]))


def key_to_dict(k):
    a, b, c, d, hk = k
    return {
        'A': list(a), 'B': list(b), 'C': list(c), 'D': list(d), 'H': dict(hk),
    }


_T1 = {'A': ['B', 'D'], 'B': ['C', 'A'], 'C': [], 'D': [None, 'D'],
       'H': {3: 'A', 5: 'B', 7: 'C'}}
_T2 = {'A': ['B', 'D'], 'B': ['C', 'A'], 'C': [], 'D': [None, 'D'],
       'H': {5: 'B', 3: 'A', 7: 'C'}}
_TK = dict_to_key(_T1)
assert _TK == dict_to_key(_T2)
assert key_to_dict(dict_to_key(_T2)) == _T2
assert dict_to_key(key_to_dict(_TK)) == _TK
_test_apply_move = apply_move(_TK, ('B', 'H', 1), 2)
_res_apply_move = {'A': ['B', 'D'], 'B': [None, 'A'], 'C': [], 'D': [None, 'D'],
                    'H': {5:'B', 3:'A', 7:'C', 1:'C'}}
assert _test_apply_move == (_res_apply_move, 4 * 100), _test_apply_move
_T3 = {}; _T3.update(_T1); _T3['C'] = [None, None]
_TK2 = dict_to_key(_T3)
_test_apply_move2 = apply_move(_TK2, ('H', 'C', 7), 2)
_res_apply_move2 = {'A': ['B', 'D'], 'B': ['C', 'A'], 'C': [None, 'C'], 'D': [None, 'D'],
                    'H': {5:'B', 3:'A'}}
assert _test_apply_move2 == (_res_apply_move2, 3*100), _test_apply_move2


def get_min_cost(status, stack_depth=2):
    goal = dict_to_key({
        'A': ['A'] * stack_depth, 'B': ['B'] * stack_depth,
        'C': ['C'] * stack_depth, 'D': ['D'] * stack_depth,
        'H': {}
    })
    pending_nodes = {dict_to_key(status)}
    min_cost = {dict_to_key(status): (0, None)}
    max_pending_nodes = 0
    while (pending_nodes):
        if len(pending_nodes) > max_pending_nodes:
            max_pending_nodes = len(pending_nodes)
        sk = pending_nodes.pop()
        status = key_to_dict(sk)
        moves = get_moves(status)
        for move in moves:
            new_status, movecost = apply_move(sk, move, stack_depth)
            nsk = dict_to_key(new_status)
            new_cost = min_cost[sk][0] + movecost
            if nsk not in min_cost or min_cost[nsk][0] > new_cost:
                min_cost[nsk] = (new_cost, )
                pending_nodes.add(nsk)
    print(f'Max pending nodes: {max_pending_nodes}, min_cost nodes: {len(min_cost)}')
    return min_cost[goal][0]


In [4]:
test1 = prob1(sample_in1)
assert test1 == 12521, test1
print('ok test')

my_res1 = prob1(my_in)
assert my_res1 > 15261, my_res1
assert my_res1 == 15299, my_res1
my_res1


Max pending nodes: 3414, min_cost nodes: 12924
Result (('B', 'A'), ('C', 'D'), ('B', 'C'), ('D', 'A')): 12521
ok test
Max pending nodes: 13269, min_cost nodes: 55639
Result (('C', 'D'), ('C', 'A'), ('B', 'B'), ('D', 'A')): 15299


15299

In [5]:
def prob2(s):
    rooms = get_input(s)
    r1, r2, r3, r4 = rooms
    status = {'A': [r1[0], 'D', 'D', r1[1]], 'B': [r2[0], 'C', 'B',r2[1]],
              'C': [r3[0], 'B', 'A', r3[1]], 'D': [r4[0], 'A', 'C',r4[1]],
              'H': {}}
    result = get_min_cost(status, 4)
    print(f'Result {rooms}: {result}')
    return result


In [6]:
test2 = prob2(sample_in1)
assert test2 == 44169, test2
assert prob2(my_in) == 47193
print('OK!')


Max pending nodes: 16693, min_cost nodes: 89105
Result (('B', 'A'), ('C', 'D'), ('B', 'C'), ('D', 'A')): 44169
Max pending nodes: 19368, min_cost nodes: 87854
Result (('C', 'D'), ('C', 'A'), ('B', 'B'), ('D', 'A')): 47193
OK!
